In [1]:
# ------------------------------------------------------------
# Imports + config
# ------------------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime


import os, sys, pathlib, importlib

from pathlib import Path
import sys

REPO_ROOT = Path.cwd().parent         
SCRIPTS_DIR = REPO_ROOT / "scripts"
if str(SCRIPTS_DIR) not in sys.path:
    sys.path.insert(0, str(SCRIPTS_DIR))

from retention_lib import (
    build_survival_and_hazard,
    merge_brand_intervals,
    derive_entry_plan,
    _normalize_event_columns,
    find_early_annual_churn,
    ALLOWED_PLANS, CAP_DATE
)

DATA_DIR = Path("../data")
RAW_EVENTS = DATA_DIR / "events.csv"              # from 00_ingest
PROC_DIR  = DATA_DIR / "processed"
PROC_DIR.mkdir(parents=True, exist_ok=True)

#CAP = pd.Timestamp("2025-04-29")
CAP = pd.Timestamp("2025-12-31")

In [2]:
# ------------------------------------------------------------
# Load normalized events (ALL plans) and build survival/hazard with audit
# ------------------------------------------------------------
events = pd.read_csv(RAW_EVENTS, parse_dates=["Start","End"])
surv_0grace, haz_0grace, audit_0grace = build_survival_and_hazard(
    events, 
    allowed_plans=ALLOWED_PLANS, 
    cap_date=CAP,
    max_months=24, 
    grace_days=0, 
    already_events=True,
    apply_exclusions=False, 
    return_audit=True
)

surv_0grace_annual = surv_0grace[surv_0grace['Plan'] == 'premium-plus'].copy()

surv_0grace_annual_2021 = surv_0grace_annual[surv_0grace_annual['Cohort Year'] == 2021].copy()
surv_0grace_annual_2022 = surv_0grace_annual[surv_0grace_annual['Cohort Year'] == 2022].copy()
surv_0grace_annual_2023 = surv_0grace_annual[surv_0grace_annual['Cohort Year'] == 2023].copy()

display(audit_0grace)        # see rows/accounts dropped/changed each stage
display(surv_0grace.head())  # sanity
display(haz_0grace.head())   # sanity   


TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and Timestamp

In [ ]:
events = pd.read_csv(RAW_EVENTS, parse_dates=["Start","End"])
surv, haz, audit = build_survival_and_hazard(
    events, 
    allowed_plans=ALLOWED_PLANS, 
    cap_date=CAP,
    max_months=24, 
    grace_days=90, 
    already_events=True,
    apply_exclusions=False, 
    return_audit=True
)

display(audit)        # see rows/accounts dropped/changed each stage
display(surv.head())  # sanity
display(haz.head())   # sanity

,stage,rows_before,rows_after,delta_rows,accounts_before,accounts_after,delta_accounts,note
0,EXCLUDE: skipped,19785,19785,0,11237,11237,0,No exclusion columns or disabled
1,NORMALIZE: raw->events,19785,19738,-47,11237,11214,-23,Normalize Start/End; cap future End; drop rows...
2,MERGE: brand windows,19785,12969,-6816,11237,11214,-23,members=11214 -> windows=12969


,Plan,Cohort Year,Month,At_Risk,Retention
0,premium-plus,2019,1,2161,1.000000
1,premium-plus,2019,2,2161,1.000000
2,premium-plus,2019,3,2155,0.997224
3,premium-plus,2019,4,2126,0.983804
4,premium-plus,2019,5,2123,0.982416


,Plan,Month,At_Risk,Exits,Avg_Hazard
0,premium-plus,1,9526,0,0.000000
1,premium-plus,2,9471,55,0.005807
2,premium-plus,3,8783,688,0.078333
3,premium-plus,4,8419,364,0.043236
4,premium-plus,5,8331,88,0.010563


In [53]:
surv_annual = surv[surv['Plan'] == 'premium-plus'].copy()

surv_annual_2021 = surv_annual[surv_annual['Cohort Year'] == 2021].copy()
surv_annual_2022 = surv_annual[surv_annual['Cohort Year'] == 2022].copy()
surv_annual_2023 = surv_annual[surv_annual['Cohort Year'] == 2023].copy()

surv_annual_2021.head(10)

,Plan,Cohort Year,Month,At_Risk,Retention
48,premium-plus,2021,1,1666,1.000000
49,premium-plus,2021,2,1655,0.993397
50,premium-plus,2021,3,1608,0.965186
51,premium-plus,2021,4,1508,0.905162
52,premium-plus,2021,5,1488,0.893157
53,premium-plus,2021,6,1412,0.847539
54,premium-plus,2021,7,1350,0.810324
55,premium-plus,2021,8,1330,0.798319
56,premium-plus,2021,9,1323,0.794118
57,premium-plus,2021,10,1304,0.782713


In [54]:
surv_0grace_annual_2021.head(10)

,Plan,Cohort Year,Month,At_Risk,Retention
48,premium-plus,2021,1,1933,1.000000
49,premium-plus,2021,2,1912,0.989136
50,premium-plus,2021,3,1862,0.963270
51,premium-plus,2021,4,1756,0.908432
52,premium-plus,2021,5,1734,0.897051
53,premium-plus,2021,6,1647,0.852043
54,premium-plus,2021,7,1579,0.816865
55,premium-plus,2021,8,1556,0.804966
56,premium-plus,2021,9,1549,0.801345
57,premium-plus,2021,10,1527,0.789964


In [55]:
surv_annual_2022.head(10)

,Plan,Cohort Year,Month,At_Risk,Retention
72,premium-plus,2022,1,911,1.000000
73,premium-plus,2022,2,901,0.989023
74,premium-plus,2022,3,891,0.978046
75,premium-plus,2022,4,878,0.963776
76,premium-plus,2022,5,874,0.959385
77,premium-plus,2022,6,865,0.949506
78,premium-plus,2022,7,858,0.941822
79,premium-plus,2022,8,856,0.939627
80,premium-plus,2022,9,853,0.936334
81,premium-plus,2022,10,852,0.935236


In [56]:
surv_annual_2023.head(10)

,Plan,Cohort Year,Month,At_Risk,Retention
96,premium-plus,2023,1,1169,1.000000
97,premium-plus,2023,2,1158,0.990590
98,premium-plus,2023,3,1080,0.923867
99,premium-plus,2023,4,1034,0.884517
100,premium-plus,2023,5,1016,0.869119
101,premium-plus,2023,6,1008,0.862275
102,premium-plus,2023,7,985,0.842601
103,premium-plus,2023,8,920,0.786997
104,premium-plus,2023,9,900,0.769889
105,premium-plus,2023,10,896,0.766467


In [57]:
# ------------------------------------------------------------
# Forensics: early annual 'churn' under brand-aware logic
# ------------------------------------------------------------
events_norm   = _normalize_event_columns(events, cap_date=CAP)
brand_windows = merge_brand_intervals(events_norm, cap_date=CAP, grace_days=0)
entry_plans   = derive_entry_plan(events_norm, allowed_plans=ALLOWED_PLANS)

early_pp = find_early_annual_churn(events_norm, brand_windows, entry_plans, months=12)

early_pp.columns

Index(['Name_or_Email', 'BrandStart', 'BrandEnd', 'Entry_Plan', 'First_Start',
       'LastMonth', 'censored', 'had_other_plan_overlap', 'other_plans'],
      dtype='object')

In [58]:
early_pp

,Name_or_Email,BrandStart,BrandEnd,Entry_Plan,First_Start,LastMonth,censored,had_other_plan_overlap,other_plans
2975,support@audiopump.co,2019-06-19 19:32:46,2019-12-09 16:01:44,premium-plus,2019-06-19 19:32:46,7,False,NaN,NaN
2321,paul+352P20June@gmail.com,2019-06-20 15:53:06,2019-12-09 16:01:44,premium-plus,2019-06-20 15:53:06,7,False,NaN,NaN
1598,juan.uno@verizon.net,2019-06-22 10:05:18,2020-01-18 08:10:04,premium-plus,2019-06-22 10:05:18,8,False,NaN,NaN
1642,katrinquosdorf@icloud.com,2019-06-23 01:35:51,2019-12-09 16:01:46,premium-plus,2019-06-23 01:35:51,7,False,False,
1165,gpietro7@gmail.com,2019-06-23 04:24:06,2019-12-22 20:24:05,premium-plus,2019-06-23 04:24:06,7,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14012,NaN,NaT,NaT,NaN,NaT,<NA>,NaN,False,
14016,NaN,NaT,NaT,NaN,NaT,<NA>,NaN,False,
14028,NaN,NaT,NaT,NaN,NaT,<NA>,NaN,False,
14032,NaN,NaT,NaT,NaN,NaT,<NA>,NaN,False,


In [59]:
print("Total early annual (<12m):", len(early_pp))
#display(early_pp.head(50))

# Split by cohort year for 2022 vs 2023
early_2021 = early_pp[early_pp["First_Start"].dt.year == 2021]
early_2022 = early_pp[early_pp["First_Start"].dt.year == 2022]
early_2023 = early_pp[early_pp["First_Start"].dt.year == 2023]
print("Early PP 2022:", len(early_2022), " | Early PP 2023:", len(early_2023))

# Save for manual QA with billing logs
#early_pp.to_csv(PROC_DIR / "early_annual_cancels_all.csv", index=False)
#early_2022.to_csv(PROC_DIR / "early_annual_cancels_2022.csv", index=False)
#early_2023.to_csv(PROC_DIR / "early_annual_cancels_2023.csv", index=False)

Total early annual (<12m): 5881
Early PP 2022: 171  | Early PP 2023: 463


In [62]:
early_2021

,Name_or_Email,BrandStart,BrandEnd,Entry_Plan,First_Start,LastMonth,censored,had_other_plan_overlap,other_plans
3334,zimakov.zimakov@gmail.com,2021-01-01 02:17:38,2021-06-01 02:17:40,premium-plus,2021-01-01 02:17:38,6,False,NaN,NaN
1738,lab90210@msn.com,2021-01-01 02:46:46,2021-04-01 02:46:49,premium-plus,2021-01-01 02:46:46,4,False,NaN,NaN
1225,hardrain77@hotmail.com,2021-01-01 06:48:57,2021-02-01 06:49:01,premium-plus,2021-01-01 06:48:57,2,False,NaN,NaN
525,christian.werle@gmx.com,2021-01-01 09:40:35,2021-03-01 09:40:36,premium-plus,2021-01-01 09:40:35,3,False,NaN,NaN
945,ericquebec@gmail.com,2021-01-01 14:03:08,2021-05-01 14:03:15,premium-plus,2021-01-01 14:03:08,5,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2676,rodandhenry@yahoo.com,2021-12-28 16:35:01,2022-02-28 16:35:53,premium-plus,2021-12-28 16:35:01,3,False,NaN,NaN
1403,jawinslow@msn.com,2020-11-15 08:17:01,2021-11-15 08:17:06,premium-plus,2021-12-28 17:04:58,0,False,False,
2762,sallymorton@bellsouth.net,2021-12-29 14:04:44,2022-07-29 14:04:46,premium-plus,2021-12-29 14:04:44,8,False,NaN,NaN
2585,rdfreson@yahoo.com,2021-12-31 00:00:00,2022-05-31 00:00:28,premium-plus,2021-12-31 00:00:00,6,False,False,
